# Tests for Normaldistribution (glm_norm)

In [1]:
import os
import datetime
import numpy as np
import pprint

import logging
import warnings

%load_ext autoreload
%autoreload 2

In [2]:
import batchglm.api as glm

np.warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning, module="tensorflow")
logging.getLogger("tensorflow").setLevel(logging.INFO)
logging.getLogger("batchglm").setLevel(logging.INFO)
logging.getLogger("diffxpy").setLevel(logging.INFO)

## Import batchglm

In [3]:
#import batchglm.api as glm

## Sample from Normal Distribution

In [4]:
norm = glm.utils.random.Normal(500, 0.7)

In [5]:
norm.sample(10)

array([500.50272376, 499.92703904, 500.00221445, 500.23249484,
       500.34491754, 499.65691853, 501.58652877, 500.32039767,
       501.23336381, 499.95040151])

## Simulate some Data

In [6]:
sim = glm.models.glm_norm.Simulator()

In [7]:
sim.generate()

In [8]:
sim.X

<xarray.DataArray 'X' (observations: 1000, features: 100)>
array([[468.891826, 513.318886, 518.586863, ..., 525.558455, 468.221244,
        462.352334],
       [466.627759, 514.130107, 520.671591, ..., 528.767221, 470.63515 ,
        461.213744],
       [467.203213, 515.406284, 519.096228, ..., 526.781062, 468.98392 ,
        463.295487],
       ...,
       [471.029414, 517.002811, 523.286155, ..., 528.02003 , 471.490833,
        463.615412],
       [469.368243, 515.44831 , 520.882381, ..., 526.577376, 469.216071,
        461.484334],
       [468.769226, 515.64164 , 522.386888, ..., 529.223504, 472.271844,
        464.378345]])
Dimensions without coordinates: observations, features

In [9]:
sim.X - sim.mean

<xarray.DataArray (observations: 1000, features: 100)>
array([[ 0.891826, -1.681114, -1.413137, ..., -0.441545, -0.778756,  0.352334],
       [-2.463706, -1.857616, -0.422457, ...,  1.71294 ,  0.548589, -1.69942 ],
       [-0.796787,  0.406284, -0.903772, ...,  0.781062, -0.01608 ,  1.295487],
       ...,
       [ 0.859768, -0.055222,  1.100753, ..., -0.099641,  0.315915, -0.280412],
       [ 0.290061, -0.622001, -0.208972, ..., -0.488014, -0.872287, -1.498326],
       [-1.400421, -1.416394,  0.201486, ...,  1.103833,  1.096926,  0.48252 ]])
Dimensions without coordinates: observations, features

In [10]:
sim.par_link_loc

<xarray.DataArray (design_loc_params: 5, features: 100)>
array([[468.      , 515.      , 520.      , ..., 526.      , 469.      ,
        462.      ],
       [  1.091465,   0.987723,   1.094049, ...,   1.054281,   1.086561,
          0.913165],
       [  0.930363,   0.979237,   1.084087, ...,   1.090288,   1.008796,
          0.940791],
       [  0.956309,   0.990543,   1.033886, ...,   1.021145,   0.940083,
          1.058845],
       [  1.078182,   1.070311,   1.091353, ...,   1.06539 ,   1.088357,
          0.98266 ]])
Coordinates:
  * design_loc_params  (design_loc_params) object 'Intercept' ... 'batch[T.3]'
Dimensions without coordinates: features

In [11]:
sim.par_link_scale

<xarray.DataArray (design_scale_params: 5, features: 100)>
array([[ 0.081509, -0.046702,  0.025192, ...,  0.047117, -0.085215,  0.000786],
       [-0.033389, -0.013031, -0.012479, ..., -0.005985, -0.076151, -0.030391],
       [-0.077662, -0.076007, -0.043802, ..., -0.01331 ,  0.04466 , -0.054178],
       [ 0.037445, -0.026021, -0.099496, ..., -0.103576, -0.05122 ,  0.063628],
       [ 0.064789,  0.020997,  0.073303, ..., -0.005952, -0.046188, -0.029621]])
Coordinates:
  * design_scale_params  (design_scale_params) object 'Intercept' ... 'batch[T.3]'
Dimensions without coordinates: features

## Set up Estimator

In [12]:
X = sim.X
design_loc = sim.design_loc
design_scale = sim.design_scale

# input data
input_data = glm.models.glm_norm.InputData.new(data=X, design_loc=design_loc, design_scale=design_scale)

In [13]:
estimator = glm.models.glm_norm.Estimator(input_data)
#estimator.initialize()

AttributeError: 'BasicModelGraph' object has no attribute 'mu'